In [1]:
import pandas as pd

In [3]:
url = 'https://raw.githubusercontent.com/rabiibouhestine/Kaggle-cat-in-the-dat/master/train.csv'
df = pd.read_csv(url)

In [3]:
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [4]:
mapping = {
"Freezing": 0,
"Warm": 1,
"Cold": 2,
"Boiling Hot": 3,
"Hot": 4,
"Lava Hot": 5
}

In [5]:
# Value counts before mapping:
df.ord_2.value_counts()

Freezing       99816
Lava Hot       63908
Boiling Hot    60627
Cold           33768
Hot            22227
Warm           19654
Name: ord_2, dtype: int64

In [6]:
df[["ord_2"]] =  df.ord_2.map(mapping)

# Value counts after mapping:
df.ord_2.value_counts()

0    99816
5    63908
3    60627
2    33768
4    22227
1    19654
Name: ord_2, dtype: int64

This type of encoding of categorical variables is known as Label Encoding, i.e.,we are encoding every category as a numerical label.

We can do the same by using LabelEncoder from scikit-learn.

In [7]:
from sklearn import preprocessing

# reat data
url = 'https://raw.githubusercontent.com/rabiibouhestine/Kaggle-cat-in-the-dat/master/train.csv'
df = pd.read_csv(url)

# fill NaN values in ord_2 column
df.loc[:, "ord_2"] = df.ord_2.fillna("NONE")

lbl_enc = preprocessing.LabelEncoder()

# fit label encoder and transform values on ord_2 column
# P.S: do not use this directly. fit first, then transform
df.loc[:, "ord_2"] = lbl_enc.fit_transform(df.ord_2.values)



In [8]:
import numpy as np
from scipy import sparse

# create our example feature matrix
example = np.array(
                [
                [0, 0, 1],
                [1, 0, 0],
                [1, 0, 1]
                ])

# convert numpy array to sparse CSR matrix
sparse_example = sparse.csr_matrix(example)



In [9]:
sparse_example

<3x3 sparse matrix of type '<class 'numpy.longlong'>'
	with 4 stored elements in Compressed Sparse Row format>

In [10]:
str(sparse_example)

'  (0, 2)\t1\n  (1, 0)\t1\n  (2, 0)\t1\n  (2, 2)\t1'

In [11]:
import numpy as np
from scipy import sparse


# number of rows
n_rows = 10000
# number of columns
n_cols = 100000

# create random binary matrix with only 5% values as 1s
example = np.random.binomial(1, p=0.05, size=(n_rows, n_cols))

# print size in bytes
print(f"Size of dense array: {example.nbytes}")
# convert numpy array to sparse CSR matrix
sparse_example = sparse.csr_matrix(example)
# print size of this sparse matrix
print(f"Size of sparse array: {sparse_example.data.nbytes}")
full_size = (
sparse_example.data.nbytes +
sparse_example.indptr.nbytes +
sparse_example.indices.nbytes
)
# print full size of this sparse matrix
print(f"Full size of sparse array: {full_size}")


Size of dense array: 8000000000
Size of sparse array: 400009648
Full size of sparse array: 600054476


In [15]:
df["new_feature"] = (df.ord_1.astype(str) + "_" + df.ord_2.astype(str))
df["new_feature"] = (df.ord_1.astype(str) + "_" + df.ord_2.astype(str))

In [16]:
df.new_feature

0         Grandmaster_1
1         Grandmaster_3
2              Expert_4
3         Grandmaster_0
4         Grandmaster_2
              ...      
299995    Contributor_2
299996         Novice_2
299997         Novice_0
299998         Master_0
299999    Contributor_2
Name: new_feature, Length: 300000, dtype: object

In [17]:
df.ord_4 = df.ord_4.fillna("NONE")

In [19]:
# We say that wherever the value count for a certain category is less than 2000, replace it with rare.
# For binary classification, it might be skewed traget - use StratifiedKFold later
df.loc[ df["ord_4"].value_counts()[df["ord_4"]].values < 2000, "ord_4"] = "RARE"

In [37]:
from sklearn import model_selection

df = pd.read_csv(url)

# we create a new column called kfold and fill it with -1
df["kfold"] = -1

# the next step is to randomize the rows of the data
df = df.sample(frac=1).reset_index(drop=True)

# fetch labels
y = df.target.values

# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f
    


In [ ]:
# df

In [38]:
# ohe_logres.py
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

def run(fold, data):
    df = data.copy()
    # load the full training data with folds
    #df = pd.read_csv(url)

    # all columns are features except id, target and kfold columns
    features = [f for f in df.columns if f not in ("id", "target", "kfold")]

    # fill all NaN values with NONE
    # note that I am converting all columns to "strings"
    # it doesn’t matter because all are categories
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")

    # get training data using folds
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # get validation data using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    # initialize OneHotEncoder from scikit-learn
    ohe = preprocessing.OneHotEncoder()

    # fit ohe on training + validation features
    full_data = pd.concat([df_train[features], df_valid[features]],axis=0)
    
    ohe.fit(full_data[features])
    
    # transform training data
    x_train = ohe.transform(df_train[features])
    
    # transform validation data
    x_valid = ohe.transform(df_valid[features])
    
    # initialize Logistic Regression model
    model = linear_model.LogisticRegression()
    
    # fit model on training data (ohe)
    model.fit(x_train, df_train.target.values)

    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)[:, 1]

    # get roc auc score
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
    
    # print auc
    print(f"Fold = {fold}, AUC = {auc}")
    
    

In [39]:
# run function for fold = 0
# we can just replace this number and
# run this for any fold
run(0,df)

Fold = 0, AUC = 0.7966309072113826


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [40]:
for fold_ in range(5):
    run(fold_, df)

/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold = 0, AUC = 0.7966309072113826


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold = 1, AUC = 0.7984243305934173


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold = 2, AUC = 0.7998085833722692


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Fold = 3, AUC = 0.797120146119528
Fold = 4, AUC = 0.7924881907525214


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC seems stable..

In [83]:
# Demo LabelEncoder
lbl = preprocessing.LabelEncoder()
vals = np.random.choice(['aa', 'alpha', 'b', 'xyz'],  size=10, replace=True, p=None).reshape(-1,1)
lbl.fit(vals)
trs = lbl.transform(vals)
print(trs)
pd.DataFrame({'orig':vals.reshape(-1), 'new': trs})

[0 0 2 0 0 1 0 0 0 0]


/Users/jvsingh/anaconda3/envs/aaml/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,orig,new
0,aa,0
1,aa,0
2,xyz,2
3,aa,0
4,aa,0
5,alpha,1
6,aa,0
7,aa,0
8,aa,0
9,aa,0


In [91]:
# Random forest
from sklearn import ensemble

def run_rf(fold, data):
    df = data.copy()
    # load the full training data with folds
    #df = pd.read_csv(url)

    # all columns are features except id, target and kfold columns
    features = [f for f in df.columns if f not in ("id", "target", "kfold")]

    # fill all NaN values with NONE
    # note that I am converting all columns to "strings"
    # it doesn’t matter because all are categories
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")
        
    # now its time to label encode the features
    for col in features:
        lbl = preprocessing.LabelEncoder()
        # fit label encoder on all data
        lbl.fit(df[col])
        # transform all the data
        df.loc[:, col] = lbl.transform(df[col])
        

    # get training data using folds
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # get validation data using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    # initialize OneHotEncoder from scikit-learn
    #ohe = preprocessing.OneHotEncoder()
    # fit ohe on training + validation features
    #full_data = pd.concat([df_train[features], df_valid[features]],axis=0)
    #
    #ohe.fit(full_data[features])
    
    # transform training data
    x_train = df_train[features].values
    
    # transform validation data
    x_valid = df_valid[features].values
    
    # initialize  model
    model = ensemble.RandomForestClassifier(n_jobs=-1)
    
    # fit model on training data (ohe)
    model.fit(x_train, df_train.target.values)

    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)[:, 1]

    # get roc auc score
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
    
    # print auc
    print(f"Fold = {fold}, AUC = {auc}")
    

In [93]:
from sklearn import model_selection

df = pd.read_csv(url)

# we create a new column called kfold and fill it with -1
df["kfold"] = -1

# the next step is to randomize the rows of the data
df = df.sample(frac=1).reset_index(drop=True)

# fetch labels
y = df.target.values

# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f
    
    
for fold_ in range(5):
    run_rf(fold_, df)

Fold = 0, AUC = 0.7333673698545807
Fold = 1, AUC = 0.732662072369133
Fold = 2, AUC = 0.737999543325451
Fold = 3, AUC = 0.7355850907849831
Fold = 4, AUC = 0.7353569438689953


The random forest model, without any tuning of hyperparameters, performs a lot worse than simple logistic regression. => Start with simple model first.

Also I saw random forest was slow and definintely more space consuming.

## SVD based

In [57]:
from sklearn import model_selection

df = pd.read_csv(url)

# we create a new column called kfold and fill it with -1
df["kfold"] = -1

# the next step is to randomize the rows of the data
df = df.sample(frac=1).reset_index(drop=True)

# fetch labels
y = df.target.values

# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f


In [ ]:
from sklearn import decomposition
from sklearn import metrics
from sklearn import preprocessing
from sklearn import ensemble
from scipy import sparse

def run_rf_svdtruncated(fold, data):
    df = data.copy()
    # load the full training data with folds
    #df = pd.read_csv(url)

    # all columns are features except id, target and kfold columns
    features = [f for f in df.columns if f not in ("id", "target", "kfold")]

    # fill all NaN values with NONE
    # note that I am converting all columns to "strings"
    # it doesn’t matter because all are categories
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")

    # get training data using folds
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # get validation data using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    
    # initialize OneHotEncoder from scikit-learn
    ohe = preprocessing.OneHotEncoder()
    # fit ohe on training + validation features
    full_data = pd.concat([df_train[features], df_valid[features]],axis=0)
    ohe.fit(full_data[features])
    
    # transform training data
    x_train = ohe.transform(df_train[features])
    
    # transform validation data
    x_valid = ohe.transform(df_valid[features])
    
    # initialize Truncated SVD
    # we are reducing the data to 120 components
    svd = decomposition.TruncatedSVD(n_components=120)

    # fit svd on full sparse training data
    full_sparse = sparse.vstack((x_train, x_valid))
    svd.fit(full_sparse)

    # transform sparse training data
    x_train = svd.transform(x_train)
    # transform sparse validation data
    x_valid = svd.transform(x_valid)

    # initialize  model
    model = ensemble.RandomForestClassifier(n_jobs=-1)
    
    # fit model on training data (ohe)
    model.fit(x_train, df_train.target.values)

    # fit model on training data (ohe)
    model.fit(x_train, df_train.target.values)
    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)[:, 1]
    # get roc auc score
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
    # print auc
    return(f"Fold = {fold}, AUC = {auc}")

In [58]:
# TIME taking - so we run in parallel
#for fold in range(5):
#    run_rf_svdtruncated(fold, df)

In [ ]:
import ray

@ray.remote
def run_rf_svdtruncated_remote(fold, data):
    return run_rf_svdtruncated(fold, data)
    
context = ray.init()
print(context.dashboard_url)

results = []
for fold in range(5):
    results.append(run_rf_svdtruncated_remote.remote(fold, df))
   
rs = ray.get(results)  # [0, 1, 2, 3]

for r in results:
    print(r)
    
ray.shutdown()


2024-04-25 22:23:29,877	INFO worker.py:1642 -- Started a local Ray instance.
